# VanillaModels

In this notebook we illustrate the properties of basic models for Vanilla options.

Vanilla options essentially depend on the terminal distribution of a reference rate. In our case the reference rate is a forward swap rate (or alternatively a Libor/Euribor rate). Distributions are calculated in the *annuity measure* such that the forward swap rate is a martingale.

In [ ]:
import sys
sys.path.append('../')

import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import lognorm
from scipy.stats import norm

We set some parameters and function used across all models.

In [ ]:
S0 = 0.0050  # 0.50% initial forward swap rate
T  = 1.0     # 1y
#
ST = np.linspace(-0.01, 0.02, 201)

In [ ]:
def plot_distributions(dists, styles, labels, title ):
    fig_size = (8, 5) # inches
    plt.figure(figsize=fig_size)
    for dist, style, label in zip(dists, styles, labels):
        plt.plot(ST, dist.pdf(ST), style, label=label)
        plt.ylabel('density')
        plt.xlabel('swap rate')
    plt.title(title)

## Normal Model

Swap rate assumed follow normal distribution,
$$
  S(T) \sim {\cal N}\left( S(t), \sigma_{\text{N}}^2 \left(T - t\right)  \right).
$$

In [ ]:
sigma_normal = 0.00313 # 31.3bp
normal_model_dist = norm(loc=S0, scale=np.sqrt(sigma_normal**2 * T) )
#
plot_distributions([normal_model_dist], ['b'], [''], 'Normal Model Distribution')

We calculate the forward price of an at-the-money call option with that Normal model.

In [ ]:
from src.helpers import bachelier

fwd_price_normal = bachelier(S0, S0, sigma_normal, T, callOrPut=1)
display(fwd_price_normal)

## Log-normal Model

We assume that log-swap rate follows normal distribution,
$$
  \ln\left(S(T)\right) \sim {\cal N}\left( 
    \ln\left(S(t)\right) - \frac{1}{2}\sigma_{\text{LN}}^2 (T-t), 
    \sigma_{\text{LN}}^2 \left(T - t\right)
    \right).
$$

In [ ]:
sigma_lognormal = 0.637  # 63.7%
lognormal_model_dist = lognorm(
    s = np.sqrt(sigma_lognormal**2 * T),
    scale = np.exp(np.log(S0) - 0.5 * sigma_lognormal**2 * T)
)
#
plot_distributions([lognormal_model_dist], ['r'], [''], 'Log-ormal Model Distribution')

We calculate the forward price of an at-the-money call option with that Log-Normal model.

In [ ]:
from src.helpers import black

fwd_price_lognormal = black(S0, S0, sigma_lognormal, T, callOrPut=1)
display(fwd_price_lognormal)

## Shifted Log-normal Model

We assume that the log of the shifted swap rate follows a normal distribution,
$$
  \ln\left(S(T) + \lambda\right) \sim {\cal N}\left( 
    \ln\left(S(t) + \lambda\right) - \frac{1}{2}\sigma_{\text{SLN}}^2 (T-t), 
    \sigma_{\text{SLN}}^2 \left(T - t\right)
    \right).
$$

In [ ]:
shift_lambda = 0.0050
sigma_shifted_lognormal = 0.315  # 31.5%
shifted_lognormal_model_dist = lognorm(
    s = np.sqrt(sigma_shifted_lognormal**2 * T),
    scale = np.exp(np.log(S0 + shift_lambda) - 0.5 * sigma_shifted_lognormal**2 * T),
    loc = -shift_lambda
)
#
plot_distributions([shifted_lognormal_model_dist], ['g'], [''], 'Shifted Log-normal Model Distribution')

We calculate the forward price of an at-the-money call option with that Shifted Log-Normal model.

In [ ]:
fwd_price_lognormal = black(S0 + shift_lambda, S0 + shift_lambda, sigma_shifted_lognormal, T, callOrPut=1)
display(fwd_price_lognormal)

## Comparison of Distributions

We compare the densities of normal, log-normal and shifted log-normal distributions.

The volatility parameters $\sigma_{\text{N}}$, $\sigma_{\text{LN}}$ and $\sigma_{\text{SLN}}$ are *calibrated* such that for all models the forward prices of at-the-money call options coincide (up to rounding differences), i.e.
$$
  \mathbb{E}_t^A\left[ \left[S(T) - S(t)\right]^+ \right] = 0.125\%.
$$  

In [ ]:
plot_distributions(
    [normal_model_dist, lognormal_model_dist, shifted_lognormal_model_dist],
    ['b', 'r', 'g'], 
    ['Normal model', 'Log-normal model', 'Shifted Log-normal model'], 
    'Comparison of Model Distributions'
    )
plt.legend()

## Implied Volatilities

Implied volatilities are an important concept to analyse market prices of Vanilla option and to compare terminal distributions of different models.

In rates markets and major currencies Normal implied volatilities are most commonly used. This is why we here also compare *Normal implied volatilities* for our three basic models for Vanilla options.

In order to calculate implied volatilities we first need reference forward option prices. We calculate call option prices using Normal, Log-normal and Shifted Log-normal model.

In [ ]:
strikes = np.linspace(-0.01, 0.02, 301)
call_prices_normal = bachelier(strikes, S0, sigma_normal, T, callOrPut=1)
call_prices_lognormal = black(strikes, S0, sigma_lognormal, T, callOrPut=1)
call_prices_shifted_lognormal = black(strikes + shift_lambda, S0 + shift_lambda, sigma_shifted_lognormal, T, callOrPut=1)

We can also plot and compare the prices from different models.

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(strikes, call_prices_normal, 'b', label='Normal model')
plt.plot(strikes, call_prices_lognormal, 'r', label='Log-normal model')
plt.plot(strikes, call_prices_shifted_lognormal, 'g', label='Shifted Log-normal model')
plt.legend()
plt.ylabel('strike')
plt.xlabel('forward price')
plt.title('Comparison of Forward Call Prices')

Howerver, comparing option prices directly often is not very helpful because it is hard to *see* differences and deduce model properties from the prices.

This is also a motivation or reason for using implied volatilities to compare option prices and models.

We said we choose *Normal implied volatilities* as our common representation of option prices. So, let us calculate the implied volatilities for the reference prices originating from our three basic models.

In [ ]:
from src.helpers import bachelier_implied_vol

normal_implied_vol_from_normal_model = np.array([
    bachelier_implied_vol(P, K, S0, T, callOrPut=1) for P, K in zip(call_prices_normal, strikes) ])

normal_implied_vol_from_lognormal_model = np.array([
    bachelier_implied_vol(P, K, S0, T, callOrPut=1) for P, K in zip(call_prices_lognormal, strikes) ])
    
normal_implied_vol_from_shifted_lognormal_model = np.array([
    bachelier_implied_vol(P, K, S0, T, callOrPut=1) for P, K in zip(call_prices_shifted_lognormal, strikes) ])

Now, we can compare the resulting implied volatilities.

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(strikes, normal_implied_vol_from_normal_model, 'b', label='Normal model')
plt.plot(strikes, normal_implied_vol_from_lognormal_model, 'r', label='Log-normal model')
plt.plot(strikes, normal_implied_vol_from_shifted_lognormal_model, 'g', label='Shifted Log-normal model')
plt.legend()
plt.ylabel('strike')
plt.xlabel('Normal implied volatilities')
plt.title('Comparison of Normal Implied Volatilities')

We comment on some observations from the **Normal** implied volatility graphs:

  - Implied volatility from Normal model is flat and coincides with the model's volatility parameter $\sigma_{\text{N}}$.
    This is because normal implied volatility inverts Bachelier formula which was used to derive reference option prices.

  - Implied volatility from Log-normal model vanishes for negative strikes and increases for increasing strikes.
    Vanishing implied volatility for negative strikes follows from the fact that the Log-normal model does not allow for
    negative strikes. The positive slope of implied volatilities follows from the higher right tail of the Log-normal
    distribution compared to the normal distribution.

  - Implied volatilities from Shifted Log-normal model show similar graph as from Log-normal model. This expected because
    Shifted Log-normal model becomes a Log-Normal model if shift $\lambda\rightarrow 0$. Implied volatilities also vanish
    for strikes less then $-\lambda$ because the model does not allow modelling of options for such strikes.
    
    Moreover, we observe some numerical instabilities for $K \approx -\lambda$. Such instabilities may occur when options
    are far in-the-money. In this example, we can avoid the instability by choosing pur reference prices instead of call
    reference prices; see below.

    In general, it is good practice to use put prices for low strikes $K<S(t)$ and call prices for high strikes $K>S(t)$
    when calculating implied volatilities.

  - For $K=S(t)$ (at-the-money) implied volatilities from all three models coincide. This follows from the fact that we
    *calibrated* all models to produce the same forward price $\mathbb{E}_t^A\left[ \left[S(T) - S(t)\right]^+ \right] = 0.125\%$
    for the at-the-money strike.

We repeat the Normal implied volatility calculation for put option prices.

In [ ]:
put_prices_normal = bachelier(strikes, S0, sigma_normal, T, callOrPut=-1)
put_prices_lognormal = black(strikes, S0, sigma_lognormal, T, callOrPut=-1)
put_prices_shifted_lognormal = black(strikes + shift_lambda, S0 + shift_lambda, sigma_shifted_lognormal, T, callOrPut=-1)
#
normal_implied_vol_from_normal_model = np.array([
    bachelier_implied_vol(P, K, S0, T, callOrPut=-1) for P, K in zip(put_prices_normal, strikes) ])

normal_implied_vol_from_lognormal_model = np.array([
    bachelier_implied_vol(P, K, S0, T, callOrPut=-1) for P, K in zip(put_prices_lognormal, strikes) ])
    
normal_implied_vol_from_shifted_lognormal_model = np.array([
    bachelier_implied_vol(P, K, S0, T, callOrPut=-1) for P, K in zip(put_prices_shifted_lognormal, strikes) ])
#
plt.figure(figsize=(8,5))
plt.plot(strikes, normal_implied_vol_from_normal_model, 'b', label='Normal model')
plt.plot(strikes, normal_implied_vol_from_lognormal_model, 'r', label='Log-normal model')
plt.plot(strikes, normal_implied_vol_from_shifted_lognormal_model, 'g', label='Shifted Log-normal model')
plt.legend()
plt.ylabel('strike')
plt.xlabel('Normal implied volatilities')
plt.title('Comparison of Normal Implied Volatilities')

We find that the numerical instability for Shifted Log-normal model disappears.

Other then that the implied volatilities from put option prices coincide with implied volatilities from call option prices. This property is another reason why to prefer implied volatilities over prices when comparing models.

## Volatility Smile Fit

We analyse the possibility to fit market quotes for Normal implied volatilities by our basic Vanilla models.

We assume forward swap rate is 0.50%. Strikes for market quotes are typically given basis points ($1bp = 10^{-4}$) and relative to the forward swap rate (at-the-money).

Smile quotes are typically given relative to the at-the-money volatility quote and also measured in basis points.

In [ ]:
relative_strikes = np.array([ -150, -100, -50, -25, 0, 25, 50, 100, 150 ]) * 1.0e-4
smile_quotes = np.array([ -3.97, -2.93, -1.73, -0.94, 0.0, 1.11, 2.39, 5.42, 9.00 ]) * 1.0e-4
atm_vol_quote = 72.02 * 1.0e-4

We can calculate the absolute strikes and volatility quotes and plot the market data.

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(S0+relative_strikes, atm_vol_quote+smile_quotes, 'm*', label='market quotes')
plt.ylim(0.0060, 0.0085)
plt.xlabel('strike')
plt.ylabel('Normal implied volatility')
plt.legend()

Now we turn to the question of how we can calibrate the parameters of our basic Vanilla models such that model-implied volatilities match the market quotes as good as possible.

With a Normal model we always get flat Normal implied volatilities. Consequently, we cannot fit all market quotes but need to *decide* at which strike level we want to match the market. The at-the-money strike is traded most liquidly. So the strike $K=S(t)$ is a reasonable choice.

Log-Normal model is not applicable for these market data. We clearly see market quotes for negative absolute strikes. Call and put options with such strikes cannot be modelled with a Log-normal model at all.

For Shifted Log-normal model we can try to find parameters $\sigma_{\text{SLN}}$ and $\lambda$ such that Normal model-implied volatilities are as close as possible to market quotes.

In order to calculate implied volatilities from a model we need to do two steps:

  1. Calculate Vanilla option price with the pricing formula for that model.

  2. Invert Bachelier formula to obtain the Normal implied volatility.

We summarise both steps in an auxiliary function.

In [ ]:
def implied_vol_from_shifted_lognormal_model(T, strike, forward, sigma_sln, shift_lambda):
    callOrPut = 1 if strike > S0 else -1
    fwd_price = black(strike + shift_lambda, S0 + shift_lambda, sigma_sln, T, callOrPut)
    return bachelier_implied_vol(fwd_price, strike, forward, T, callOrPut)

Now, we can plot and compare Normal model volatilities and Shifted Log-normal model volatilities with our market quotes.

In [ ]:
implied_vol_n = atm_vol_quote * np.ones(len(strikes))

sigma_sln = 0.085
shift_lambda = 0.08
implied_vol_sln = np.array([ implied_vol_from_shifted_lognormal_model(5.0, K, S0, sigma_sln, shift_lambda) for K in strikes ])

plt.figure(figsize=(8,5))
plt.plot(S0+relative_strikes, atm_vol_quote+smile_quotes, 'm*', label='market quotes')
plt.plot(strikes, implied_vol_n, 'b', label='Normal model')
plt.plot(strikes, implied_vol_sln, 'g', label='Shifted Log-normal model')
plt.ylim(0.0060, 0.0085)
plt.xlabel('strike')
plt.ylabel('Normal implied volatility')
plt.legend()

For this example we can fit a Shifted Log-normal model to the at-the-money volatility quote and the slope (or *volatility skew*) of  market implied volatilities around the at-the-money strike. The resulting Shifted Log-normal model parameters are $\sigma_{\text{SLN}}=8.5\%$ and $\lambda=8\%$.

However, the market quotes also exhibit some curvature or *volatility smile*. This property can not be captured by our basic Vanilla models.